In [1]:
import pandas as pd
import requests
import boto3
import ast
import datetime
import time

In [2]:
from cobinhood_api import Cobinhood
cob = Cobinhood()

print(cob.system.get_time())

2018-07-27 07:16:54 Chriss-MacBook-Pro.local cobinhood-api[4018] DEBUG cobinhood_api.http.system fetch "get_time"


{'success': True, 'result': {'time': 1532690215128}}


In [3]:
cob.market.get_trading_pairs()

2018-07-27 07:16:56 Chriss-MacBook-Pro.local cobinhood-api[4018] DEBUG cobinhood_api.http.market fetch "get_trading_pairs"


{'success': True,
 'result': {'trading_pairs': [{'id': 'ABT-BTC',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'BTC',
    'base_max_size': '2516723.629',
    'base_min_size': '50.335',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'ABT-ETH',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'ETH',
    'base_max_size': '2516723.629',
    'base_min_size': '50.335',
    'quote_increment': '0.0000001',
    'margin_enabled': False},
   {'id': 'ACT-BTC',
    'base_currency_id': 'ACT',
    'quote_currency_id': 'BTC',
    'base_max_size': '11934470.211',
    'base_min_size': '238.69',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-BTC',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'BTC',
    'base_max_size': '2991280.418',
    'base_min_size': '59.826',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-ETH',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'ETH',

In [4]:
# Instantiate resources
try:
    boto_session = boto3.Session(profile_name='loidsig')
except:
    boto_session = boto3.Session()
s3_resource = boto_session.resource('s3')
s3_bucket = 'loidsig-crypto'

sm_client = boto_session.client(
    service_name='secretsmanager',
    region_name='us-east-1',
    endpoint_url='https://secretsmanager.us-east-1.amazonaws.com'
)
get_secret_value_response = sm_client.get_secret_value(SecretId='Loidsig_CPM_Cobinhood')
key, value = ast.literal_eval(get_secret_value_response['SecretString']).popitem()
cob = Cobinhood(API_TOKEN=value)

In [10]:
print(cob.wallet.get_balances())

2018-07-24 08:04:30 Chriss-MacBook-Pro.local cobinhood-api[1098] DEBUG cobinhood_api.http.wallet fetch "get_balances"


{'success': True, 'result': {'balances': [{'currency': 'ETH', 'type': 'exchange', 'total': '1.49', 'on_order': '0', 'locked': False, 'usd_value': '707.81407', 'btc_value': '0.08652877'}]}}


In [18]:
cob_candles_list = cob.chart.get_candles(trading_pair_id='ETH-USDT', start_time=1519862400000, end_time=1519948800000, timeframe='1m')['result']['candles']
df = pd.DataFrame(cob_candles_list)
df.rename(columns={'trading_pair_id':'coin','timestamp':'close_timestamp'}, inplace=True)
df['coin'] = df['coin'].replace(regex=True, to_replace=r'-', value=r'')
df['close_timestamp_trim'] = df['close_timestamp']/1000
df['close_datetime'] = pd.to_datetime(df['close_timestamp_trim'], unit='s')

print(f"""Current: {datetime.datetime.utcfromtimestamp(time.time()).strftime('%Y-%m-%d %I:%M:%S')}
Start: {datetime.datetime.utcfromtimestamp(1519516800).strftime('%Y-%m-%d %I:%M:%S')}""")
df

2018-07-27 07:27:47 Chriss-MacBook-Pro.local cobinhood-api[4018] DEBUG cobinhood_api.http.chart fetch "get_candles"


Current: 2018-07-27 11:27:47
Start: 2018-02-25 12:00:00


,close,high,low,open,timeframe,close_timestamp,coin,volume,close_timestamp_trim,close_datetime
0,860,860,860,860,1m,1519862400000,ETHUSDT,0,1.519862e+09,2018-03-01 00:00:00
1,860,860,860,860,1m,1519862460000,ETHUSDT,0,1.519862e+09,2018-03-01 00:01:00
2,860,860,860,860,1m,1519862520000,ETHUSDT,0,1.519863e+09,2018-03-01 00:02:00
3,860,860,860,860,1m,1519862580000,ETHUSDT,0,1.519863e+09,2018-03-01 00:03:00
4,860,860,860,860,1m,1519862640000,ETHUSDT,0,1.519863e+09,2018-03-01 00:04:00
5,860,860,860,860,1m,1519862700000,ETHUSDT,0,1.519863e+09,2018-03-01 00:05:00
6,860,860,860,860,1m,1519862760000,ETHUSDT,0,1.519863e+09,2018-03-01 00:06:00
7,860,860,860,860,1m,1519862820000,ETHUSDT,0,1.519863e+09,2018-03-01 00:07:00
8,860,860,860,860,1m,1519862880000,ETHUSDT,0,1.519863e+09,2018-03-01 00:08:00
9,860,860,860,860,1m,1519862940000,ETHUSDT,0,1.519863e+09,2018-03-01 00:09:00


In [21]:
df[df['volume'] != '0']

,close,high,low,open,timeframe,close_timestamp,coin,volume,close_timestamp_trim,close_datetime
132,879.9,879.9,870,870,1m,1519870320000,ETHUSDT,1.12,1.519870e+09,2018-03-01 02:12:00
156,899.9,899.9,879.9,879.9,1m,1519871760000,ETHUSDT,0.6469286,1.519872e+09,2018-03-01 02:36:00
191,899.9,899.9,899.9,899.9,1m,1519873860000,ETHUSDT,0.31932385,1.519874e+09,2018-03-01 03:11:00
315,840,855.1,840,855.1,1m,1519881300000,ETHUSDT,1,1.519881e+09,2018-03-01 05:15:00
321,838.15,840,838.15,840,1m,1519881660000,ETHUSDT,1,1.519882e+09,2018-03-01 05:21:00
324,825,838.15,825,838.15,1m,1519881840000,ETHUSDT,1.48,1.519882e+09,2018-03-01 05:24:00
647,825,836.1,825,836.1,1m,1519901220000,ETHUSDT,1,1.519901e+09,2018-03-01 10:47:00
880,875,875,875,875,1m,1519915200000,ETHUSDT,0.05991693,1.519915e+09,2018-03-01 14:40:00
881,898.8,898.8,898.8,898.8,1m,1519915260000,ETHUSDT,0.04792709,1.519915e+09,2018-03-01 14:41:00
901,840,840,840,840,1m,1519916460000,ETHUSDT,0.098,1.519916e+09,2018-03-01 15:01:00


In [17]:
1519862400+86400

1519948800